In [12]:
import pandas as pd
import numpy as np
import pickle

dict_results = {}
for ds in ['Salinas']:# ['IndianPines', 'Salinas']:
    for dr_method in ['PCA']: #['PCA', 'FastICA']:
        for n_components in [8]: #[8, 15]:
            for model in ['SGD']: #['SVM', 'SGD', 'hamida', 'li']:
                filename = f'{ds}_{dr_method}_{n_components}_{model}_50epochs.csv'
                print('\n' + filename)

                try:
                    df = pd.read_csv(f'../Results/{filename}', index_col=0)
                    print('Loaded.')
                    try:
                        df['Confusion matrix'] = df['Confusion matrix'].apply(lambda x: pickle.loads(eval(x)))
                        df['F1 scores'] = df['F1 scores'].apply(lambda x: pickle.loads(eval(x)))
                        df['IOU'] = df['IOU'].apply(lambda x: pickle.loads(eval(x)))
                        dict_results[f'{ds}_{dr_method}_{n_components}_{model}'] = df
                        print('Columns unpickled.')
                    except:
                        print("Problem with pickled columns")
                except:
                    print("Couldn't find CSV file.")
                
                


Salinas_PCA_8_SGD_50epochs.csv
Loaded.
Columns unpickled.


In [13]:
SALINAS_LABELS = [
    "Undefined",
    "Brocoli_green_weeds_1",
    "Brocoli_green_weeds_2",
    "Fallow",
    "Fallow_rough_plow",
    "Fallow_smooth",
    "Stubble",
    "Celery",
    "Grapes_untrained",
    "Soil_vinyard_develop",
    "Corn_senesced_green_weeds",
    "Lettuce_romaine_4wk",
    "Lettuce_romaine_5wk",
    "Lettuce_romaine_6wk",
    "Lettuce_romaine_7wk",
    "Vinyard_untrained",
    "Vinyard_vertical_trellis",
]

INDIANPINES_LABELS = [
    "Undefined",
    "Alfalfa",
    "Corn-notill",
    "Corn-mintill",
    "Corn",
    "Grass-pasture",
    "Grass-trees",
    "Grass-pasture-mowed",
    "Hay-windrowed",
    "Oats",
    "Soybean-notill",
    "Soybean-mintill",
    "Soybean-clean",
    "Wheat",
    "Woods",
    "Buildings-Grass-Trees-Drives",
    "Stone-Steel-Towers",
]

In [14]:
import plotly.express as px

def show_confusion_matrix(confusion_matrix, labels):
    fig = px.imshow(confusion_matrix, text_auto=True, 
                    labels=dict(x='Predicted labels', y='Actual labels', color='# of classification'),
                    x = labels,
                    y = labels,
                    height=700
                    )
    fig.update_xaxes(tickangle=45)
    fig.show()

conf_matrix = dict_results['IndianPines_FastICA_8_hamida']['Confusion matrix'][0]
show_confusion_matrix(conf_matrix, labels=SALINAS_LABELS)


KeyError: 'IndianPines_FastICA_8_hamida'

In [4]:
arr = np.array([[exp, dict_results[exp]['Accuracy'][0]] for exp in dict_results])
sorted_data = arr[np.argsort(arr[:, 1])]


fig = px.bar({'Overall Accuracy':[data[1] for data in sorted_data],
              'Experiment': [data[0] for data in sorted_data]}, x='Experiment', y='Overall Accuracy',
              width=700)
fig.update_xaxes(tickangle=45)
fig.show()

In [5]:
arr = np.array([[exp, dict_results[exp]['Kappa'][0]] for exp in dict_results])
sorted_data = arr[np.argsort(arr[:, 1])]


fig = px.bar({'Kappa':[data[1] for data in sorted_data],
              'Experiment': [data[0] for data in sorted_data]}, x='Experiment', y='Kappa',
              width=700)
fig.update_xaxes(tickangle=45)
fig.show()

In [18]:
import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name=exp, x=SALINAS_LABELS, y=dict_results[exp]['F1 scores'][0])
    for exp in dict_results
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()